In [1]:
!ls

Parser.ipynb  README.md  WNdb-3.0.tar.gz  dict	parsers


In [60]:
def createIndexDF(filePath):
    with open(filePath) as f:
        Indexdump = f.readlines()
    indexLines = []
    for i in Indexdump:
        if i[0] == " " or i[0] == ".":
            continue
        else:
            indexLines.append(i.strip())
    for i in range(len(indexLines)):
        splitLine = indexLines[i].split()
        if (len(splitLine) == 8):
            indexLines[i] = splitLine
            continue
        ls = []
        ls.extend(splitLine[0:4])
        p_cnt = int(ls[-1])
        ls.append(splitLine[4:4 + p_cnt])
        sense_cntIndex = 4 + p_cnt
        ls.append(splitLine[sense_cntIndex])
        tagsense_cntIndex = sense_cntIndex + 1
        ls.append(splitLine[tagsense_cntIndex])
        remainingStartIndex = tagsense_cntIndex + 1
        ls.append(splitLine[remainingStartIndex:])
        indexLines[i] = ls
    df = pd.DataFrame(indexLines, columns = ['lemma', 'pos', 'synset_cnt', 'p_cnt', 'ptr_symbol', 'sense_cnt', 'tagsense_cnt', 'synset_offset'])
    return df

In [264]:
def createDataDF(filePath, Isverb=False):
    with open(filePath) as f:
        Datadump = f.readlines()
    dataLines = []
    for i in Datadump:
        if i[0] == " " or i[0] == ".":
            continue
        else:
            dataLines.append(i.strip())
    for i in range(len(dataLines)):
        splitLine = dataLines[i].split()
        ls = []
        ls.extend(splitLine[:4])
        wordCount = int(ls[-1], 16)
        currIndex = 4
        ls_words_and_lexids = []
        for j in range(wordCount):
            ls_words_and_lexids.append([splitLine[currIndex],splitLine[currIndex+1]])
            currIndex += 2
        ls.append(ls_words_and_lexids)
        p_cnt = splitLine[currIndex]
        ls.append(p_cnt)
        currIndex += 1
        ptrList = []
        for j in range(int(p_cnt)):
            ptrList.append([splitLine[currIndex],splitLine[currIndex+1],splitLine[currIndex+2],splitLine[currIndex+3]])
            currIndex += 4
        ls.append(ptrList)
        if (Isverb):
            f_cnt = int(splitLine[currIndex])
            currIndex += 2
            ls.append(f_cnt)
            ls_fnum_wnum = []
            for j in range(f_cnt):
                ls_fnum_wnum.append([splitLine[currIndex],splitLine[currIndex+1]])
                currIndex += 3
            ls.append(ls_fnum_wnum)
        ls.append(" ".join(splitLine[currIndex:]))
        dataLines[i] = ls
    if (Isverb):
        df = pd.DataFrame(dataLines, columns = ['synset_offset', 'lex_filenum', 'ss_type', 'w_cnt', 'word with lex_id', 'p_cnt', 'ptr', 'f_cnt', 'frame', 'gloss'])
    else:
        df = pd.DataFrame(dataLines, columns = ['synset_offset', 'lex_filenum', 'ss_type', 'w_cnt', 'word with lex_id', 'p_cnt', 'ptr', 'gloss'])
    return df

In [266]:
createDataDF('dict/data.adj', False)

,synset_offset,lex_filenum,ss_type,w_cnt,word with lex_id,p_cnt,ptr,gloss
0,00001740,00,a,01,"[[able, 0]]",005,"[[=, 05200169, n, 0000], [=, 05616246, n, 0000...",| (usually followed by `to') having the necess...
1,00002098,00,a,01,"[[unable, 0]]",002,"[[=, 05200169, n, 0000], [!, 00001740, a, 0101]]",| (usually followed by `to') not having the ne...
2,00002312,00,a,02,"[[abaxial, 0], [dorsal, 4]]",002,"[[;c, 06037666, n, 0000], [!, 00002527, a, 0101]]",| facing away from the axis of an organ or org...
3,00002527,00,a,02,"[[adaxial, 0], [ventral, 4]]",002,"[[;c, 06037666, n, 0000], [!, 00002312, a, 0101]]",| nearest to or facing toward the axis of an o...
4,00002730,00,a,01,"[[acroscopic, 0]]",002,"[[;c, 06066555, n, 0000], [!, 00002843, a, 0101]]",| facing or on the side toward the apex
...,...,...,...,...,...,...,...,...
18151,03154786,44,a,01,"[[transpiring, 0]]",001,"[[<, 02066757, v, 0101]]","| that is passing through; ""transpiring gas"""
18152,03154886,44,a,01,"[[sought, 0]]",001,"[[<, 01315613, v, 0102]]","| that is looked for; ""the long sought relatives"""
18153,03154986,44,a,01,"[[closed-captioned, 0]]",002,"[[;c, 06277280, n, 0000], [<, 02323870, v, 0101]]",| broadcast with captions that are seen only o...
18154,03155193,44,a,01,"[[saponified, 0]]",002,"[[!, 03155306, a, 0101], [<, 00538571, v, 0101]]","| converted into soap; ""saponified oils"""
